In [1]:
!pip install azureml-core

     |████████████████████████████████| 2.2 MB 33.2 MB/s 
     |████████████████████████████████| 55 kB 4.3 MB/s 
     |████████████████████████████████| 547 kB 62.5 MB/s 
     |████████████████████████████████| 145 kB 60.2 MB/s 
     |████████████████████████████████| 796 kB 35.4 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 141 kB 59.3 MB/s 
     |████████████████████████████████| 3.0 MB 60.2 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 314 kB 62.2 MB/s 
     |████████████████████████████████| 1.3 MB 40.3 MB/s 
     |████████████████████████████████| 40 kB 5.7 MB/s 
     |████████████████████████████████| 85 kB 4.7 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 166 kB 62.7 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
     |████████████████████████████████| 45 kB 3.6 MB/s 
     |██████████████████████

In [3]:
from azureml.core import Workspace
ws = Workspace.from_config(path="config.json")
print(ws)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code A3NBLUCWH to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace.create(name='msa', subscription_id='44f273d4-60b2-41b1-b668-f43ee0dd0737', resource_group='msa')


WIll get 401 error if Kaggle API is not valid:

In [4]:
import os 
if not os.path.exists('/root/.kaggle/'):
    os.makedirs('/root/.kaggle/')
!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d matthewabilton/finetuned-bert-readability
!unzip finetuned-bert-readability.zip

 99% 234M/236M [00:01<00:00, 156MB/s]
100% 236M/236M [00:01<00:00, 143MB/s]
Archive:  finetuned-bert-readability.zip
  inflating: best_model/config.json  
  inflating: best_model/pytorch_model.bin  
  inflating: tokenizer/special_tokens_map.json  
  inflating: tokenizer/tokenizer_config.json  
  inflating: tokenizer/vocab.txt     


In [5]:
from azureml.core.model import Model
model_name = 'bertreadability'
model_path = "."
child_paths = ["best_model", "tokenizer"]
model = Model.register(ws, model_name=model_name, model_path=model_path, child_paths=child_paths)

Registering model bertreadability


Code we need to write to entry script:

In [31]:
# Entry script to write to file:
entry_script_str = \
"""
import json
import os
import torch
from torch.nn.utils.rnn import pad_sequence
import transformers
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

NUM_PRED = 1

def init():
  global model, tokenizer
  tokenizer_dir = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "content/tokenizer")
  tokenizer = tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_dir)
  model_dir = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "content/best_model")
  model = DistilBertForSequenceClassification.from_pretrained(model_dir)
  if model.training:
    model.eval()
 
def process_chunks(id_chunk, mask_chunk, max_len, start_id=101, end_id=102):
  id_chunk = torch.cat([torch.tensor([start_id]), id_chunk, torch.tensor([end_id])])
  mask_chunk = torch.cat([torch.ones(1), mask_chunk, torch.ones(1)])
  if len(id_chunk) < max_len:
    pad_len = max_len - len(id_chunk)
    id_chunk = torch.cat([id_chunk, torch.zeros(pad_len)])
    mask_chunk = torch.cat([mask_chunk, torch.zeros(pad_len)])
  return (id_chunk, mask_chunk)
 
def preprocess_text(text_list, max_len=250):
  X_id, X_mask = [], []
  for text in text_list:
    id_list, mask_list = [], []
    tokens = tokenizer.encode_plus(text, add_special_tokens=False, return_tensors='pt')
    ids = tokens["input_ids"][0].split(max_len-2)
    masks = tokens["attention_mask"][0].split(max_len-2)
    for id_chunk, mask_chunk in zip(ids, masks):
      id_chunk, mask_chunk = process_chunks(id_chunk, mask_chunk, max_len)
      id_list.append(id_chunk), mask_list.append(mask_chunk)
    X_id.append(torch.stack(id_list, dim=0))
    X_mask.append(torch.stack(mask_list, dim=0))
  X_id = pad_sequence(X_id, batch_first=True, padding_value=0)
  X_mask = pad_sequence(X_mask, batch_first=True, padding_value=0)
  X_id, X_mask = torch.as_tensor(X_id, dtype=torch.int64), torch.as_tensor(X_mask, dtype=torch.int64)
  X_wts = torch.sum(X_mask, axis=2, keepdim=True)/torch.sum(X_mask, axis=(1,2), keepdim=True)
  X_wts = X_wts.reshape(X_mask.shape[0:-1])
  X_wts = torch.as_tensor(X_wts, dtype=torch.float32)
  return (X_id, X_mask, X_wts)
 
def make_pred(X_id, X_mask, X_wts):
  with torch.no_grad():
    in_shape = (X_id.shape[0]*X_id.shape[1], X_id.shape[2])
    out_shape = X_id.shape[0:-1] + (NUM_PRED,)
    X_id, X_mask = X_id.reshape(in_shape), X_mask.reshape(in_shape)
    logits = model(input_ids=X_id, attention_mask=X_mask)["logits"]
    logits = logits.reshape(out_shape)
    pred = torch.einsum("ij,ijk->ik", X_wts, torch.nan_to_num(logits))
  return pred
 
def run(request):
  req_json = json.loads(request)
  text_list = req_json["text"]
  X_id, X_mask, X_wts = preprocess_text(text_list)
  pred = make_pred(X_id, X_mask, X_wts)
  output = [x.item() for x in pred]
  return output
"""

# Write entry script string to Python file:
entry_script_dir = "entry_script.py"
with open(entry_script_dir, 'w') as f: 
  f.write(entry_script_str)

In [32]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig

env_name = "bert-env"
env = Environment(name=env_name)
conda_dep = CondaDependencies()
conda_dep.add_pip_package("pip")
conda_dep.add_pip_package("torch")
conda_dep.add_pip_package("transformers")
env.python.conda_dependencies = conda_dep

inference_config = InferenceConfig(environment=env,
                                   entry_script=entry_script_dir)

In [33]:
from azureml.core.webservice import AciWebservice
num_cores = 1
gb_mem = 2
aci_config = AciWebservice.deploy_configuration(cpu_cores=num_cores, 
                                                memory_gb=gb_mem)
service = Model.deploy(ws,
                       env_name,
                       [model],
                       inference_config,
                       aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-15 00:07:27+00:00 Registering the environment.
2021-09-15 00:07:29+00:00 Use the existing image.
2021-09-15 00:07:30+00:00 Generating deployment configuration.
2021-09-15 00:07:31+00:00 Submitting deployment to compute.
2021-09-15 00:07:38+00:00 Checking the status of deployment bert-env..
2021-09-15 00:15:02+00:00 Checking the status of inference endpoint bert-env.
Succeeded
ACI service creation operation finished, operation "Succeeded"


Try call API with a few example pieces of text:

In [37]:
import requests
import json

uri = service.scoring_uri
# Paste in the API end point of our deployed service:
end_point = 'http://d7cd4e84-d0f9-4d93-8de7-aba2c62ef9bb.australiaeast.azurecontainer.io/score'
# Initial call to API:
requests.get(end_point)
# Create input text data to POST to API:
headers = {"Content-Type": "application/json"}
data = {"text": ["Mr. Bennet was among the earliest of those who waited on Mr. Bingley. He had always intended to visit him, though to the last always assuring his wife that he should not go; and till the evening after the visit was paid she had no knowledge of it. It was then disclosed in the following manner.",
                 "So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.",
                 "I am already far north of London, and as I walk in the streets of Petersburgh, I feel a cold northern breeze play upon my cheeks, which braces my nerves and fills me with delight. Do you understand this feeling? This breeze, which has travelled from the regions towards which I am advancing, gives me a foretaste of those icy climes. Inspirited by this wind of promise, my daydreams become more fervent and vivid. I try in vain to be persuaded that the pole is the seat of frost and desolation; it ever presents itself to my imagination as the region of beauty and delight. There, Margaret, the sun is for ever visible, its broad disk just skirting the horizon and diffusing a perpetual splendour. There—for with your leave, my sister, I will put some trust in preceding navigators—there snow and frost are banished; and, sailing over a calm sea, we may be wafted to a land surpassing in wonders and in beauty every region hitherto discovered on the habitable globe. Its productions and features may be without example, as the phenomena of the heavenly bodies undoubtedly are in those undiscovered solitudes. What may not be expected in a country of eternal light?"]}
data = json.dumps(data)
# Send POST request to API:
response = requests.post(uri, data=data, headers=headers)
# Print predictions of our network:
print(response.json())

[-0.6307001113891602, -0.23245826363563538, -2.040571689605713]
